## EDA: in interations, too
Can't know it all at a first glance.

In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)  # Check CUDA version (if using GPU)
# 2.5.1+cu121
# 12.1

2.5.1+cu121
12.1


In [2]:
import os

import numpy as np

# Image processing
import tifffile
import matplotlib.pyplot as plt
# from PIL import Image
# import cv2


In [5]:
WORKING_DIR = "/Users/thuang/Documents/Personal/code/prostate-cancer-grade-assessment"
# /kaggle/working

os.chdir(WORKING_DIR)
print(os.getcwd())

DATA_ROOT_PATH = "data"
# "/kaggle/input/prostate-cancer-grade-assessment"

/Users/thuang/Documents/Personal/code/prostate-cancer-grade-assessment


### Questions
* Data quality, distribution, outliers?
* What to use? What to disgard? What/how to preprocess/augment?
* Valid sub-sample to download and test initial models on?

#### Whole Sliding Image(WSI)
*

#### Image size, resolution, tiff stack
* Size, resolution varies?

In [6]:
filename = os.path.join(DATA_ROOT_PATH, "train_images/0005f7aaab2800f6170c399693a96917.tiff")

with tifffile.TiffFile(filename) as tiff:
    page = tiff.pages[0]  # Access the first page of the TIFF file
    width, height = page.shape[:2]
    dpi_x, dpi_y = page.tags[282].value, page.tags[283].value  # XResolution & YResolution
    dpi_unit = page.tags[296]
print(dpi_x, dpi_y, dpi_unit)

(22123, 1) (22123, 1) TiffTag 296 ResolutionUnit @43318386 SHORT @43318398 = CENTIMETER


#### Visualize image examples to inspect